## Coursera Capstone - Neighborhood Clustering Pt.3

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### Data import and pre-processing:

In [2]:
ctable = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
ctable = ctable[0]
ctable = ctable[ctable['Borough'] != 'Not assigned']
ctable = ctable.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()

Getting coords from CSV and merging with the main table:

In [3]:
coords = pd.read_csv('http://cocl.us/Geospatial_data')
ctable_coord = pd.merge(ctable,coords,left_on = 'Postcode',right_on='Postal Code')
ctable_coord = ctable_coord.drop(columns=['Postal Code'])
ctable_coord

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


### Fetching venues from Foursquare

In [4]:
CLIENT_ID = '' #Foursquare ID removed for privacy reasons
CLIENT_SECRET = '' #Foursquare Secret removed for privacy reasons
VERSION = '20180605'

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=128): #500 meters radius for simplicity (~5 blocks), top 128 is arbitrary
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
toronto_Locs = getNearbyVenues(names=ctable_coord['Neighbourhood'],
                                   latitudes=ctable_coord['Latitude'],
                                   longitudes=ctable_coord['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [7]:
print(toronto_Locs.shape)
toronto_Locs.head() 

(2271, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Venues look good! Quite a few of them too.

### Data preparation

Now preparing the data for clustering analysis, based on the types of venues located in that postal code region:

In [8]:
toronto_Venues = pd.get_dummies(toronto_Locs[['Venue Category']], prefix="", prefix_sep="")
toronto_Venues['Neighborhood'] = toronto_Locs['Neighborhood'] 
fixed_columns = [toronto_Venues.columns[-1]] + list(toronto_Venues.columns[:-1])
toronto_Venues =toronto_Venues[fixed_columns]
toronto_Venues.shape

(2271, 273)

In [9]:
toronto_Venues = toronto_Venues.groupby('Neighborhood').mean().reset_index()
toronto_Venues.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Creating a DF with the 10 most common venues in a postal code region/"Neighborhood", for analysis down the road:

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

toronto_Top10 = pd.DataFrame(columns=columns)
toronto_Top10['Neighborhood'] = toronto_Venues['Neighborhood']

for ind in np.arange(toronto_Venues.shape[0]):
    toronto_Top10.iloc[ind, 1:] = return_most_common_venues(toronto_Venues.iloc[ind, :], 10)

toronto_Top10

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,American Restaurant,Thai Restaurant,Restaurant,Hotel,Cosmetics Shop
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Sculpture Garden,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Pharmacy,Concert Hall
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pub,Diner,Department Store
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Shopping Mall,Deli / Bodega,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Liquor Store,Indian Restaurant,Japanese Restaurant,Fast Food Restaurant,Juice Bar,Breakfast Spot,Sandwich Place,Sushi Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Italian Restaurant,Bakery,Beer Bar,Seafood Restaurant,Steakhouse,Farmers Market,Café
9,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Colombian Restaurant


Looks good! 10 venues should be enough to get a sense of what's in each neighborhood without cluttering the screen.

### Clustering and analysis

For cities, there are three broad types of zones: residential, commercial, and industrial. We can also assume a fourth type, "entertainment" - thus, let's use k = 4 as a reference and see what comes up during clustering.

In [11]:
kclust = 4

toronto_Clustered = toronto_Venues.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclust, random_state=42).fit(toronto_Clustered)

Checking what cluster each of the neighborhoods belong to:

In [12]:
kmeans.labels_

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=int32)

Hmm...most of the neighborhoods belong to cluster 0, while only a few belong to 1 and only one to 2 and 3 apiece.

Adding the cluster labels to the top 10 dataframe:

In [64]:
toronto_Top10.insert(0, 'Cluster', kmeans.labels_)
tempdf = toronto_Top10.merge(ctable_coord[['Borough','Neighbourhood','Latitude','Longitude']], left_on = 'Neighborhood',right_on='Neighbourhood')
toronto_Top10.insert(1,'Borough',tempdf['Borough'])
toronto_Top10.insert(3,'Latitude',tempdf['Latitude'])
toronto_Top10.insert(4,'Longitude',tempdf['Longitude'])
toronto_Top10.head()

,Cluster,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,Coffee Shop,Café,Bar,Steakhouse,Burger Joint,American Restaurant,Thai Restaurant,Restaurant,Hotel,Cosmetics Shop
1,0,Scarborough,Agincourt,43.794200,-79.262029,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,1,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",43.815252,-79.284577,Park,Playground,Sculpture Garden,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437,Grocery Store,Liquor Store,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Pharmacy,Concert Hall
4,0,Etobicoke,"Alderwood,Long Branch",43.602414,-79.543484,Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pub,Diner,Department Store


So, what do the clusters look like?

Cluster 0:

In [55]:
toronto_Top10.loc[toronto_Top10['Cluster'] == 0]

,Cluster,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Downtown Toronto,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Burger Joint,American Restaurant,Thai Restaurant,Restaurant,Hotel,Cosmetics Shop
1,0,Scarborough,Agincourt,Lounge,Skating Rink,Breakfast Spot,Clothing Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,0,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Liquor Store,Coffee Shop,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pizza Place,Pharmacy,Concert Hall
4,0,Etobicoke,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Gym,Pharmacy,Skating Rink,Sandwich Place,Dance Studio,Pub,Diner,Department Store
5,0,North York,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Shopping Mall,Deli / Bodega,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner
6,0,North York,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,0,North York,"Bedford Park,Lawrence Manor East",Italian Restaurant,Coffee Shop,Liquor Store,Indian Restaurant,Japanese Restaurant,Fast Food Restaurant,Juice Bar,Breakfast Spot,Sandwich Place,Sushi Restaurant
8,0,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Italian Restaurant,Bakery,Beer Bar,Seafood Restaurant,Steakhouse,Farmers Market,Café
9,0,Scarborough,"Birch Cliff,Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Colombian Restaurant
10,0,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Convenience Store,Pharmacy,Beer Store,Coffee Shop,Pizza Place,Café,Pet Store,Liquor Store,Electronics Store,Empanada Restaurant


Neighborhoods in this cluster all seem to have lots of restaurants and food, shopping, hotels, and various forms of transports - it seems like these are commercial and/or mixed neighborhoods.

Cluster 1:

In [57]:
toronto_Top10.loc[toronto_Top10['Cluster'] == 1]

,Cluster,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,1,Scarborough,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Sculpture Garden,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
16,1,York,Caledonia-Fairbanks,Park,Women's Store,Market,Fast Food Restaurant,Comic Shop,Concert Hall,Event Space,Ethiopian Restaurant,Comfort Food Restaurant,Empanada Restaurant
63,1,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Dessert Shop
73,1,North York,Parkwoods,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
74,1,Downtown Toronto,Rosedale,Park,Playground,Building,Trail,Eastern European Restaurant,Electronics Store,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
80,1,North York,"Silver Hills,York Mills",Park,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
89,1,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",River,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
93,1,York,Weston,Park,Convenience Store,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
99,1,North York,York Mills West,Park,Bank,Bar,Convenience Store,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop


Many of the top venues appear to be parks, playgrounds, small shops, etc. - this suggests that the neighborhoods in cluster 1 are primarily residential. In fact, looking up some of the neighborhoods in this cluster, like Rosedale, seem to confirm these findings.

Cluster 2:

In [58]:
toronto_Top10.loc[toronto_Top10['Cluster'] == 2]

,Cluster,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,2,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


With only one result in this cluster, its hard to decide what this neighborhood fits in. Searching online suggests a suburban commerical district.

Cluster 3:

In [60]:
toronto_Top10.loc[toronto_Top10['Cluster'] == 3]

,Cluster,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,3,Central Toronto,Roselawn,Garden,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store


Same as with cluster 2, I would guess another residential-focused or mixed district?

Last but not least, let's visualize these clusters on a map:

In [67]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
x = np.arange(kclust)
ys = [i + x + (i*x)**2 for i in range(kclust)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_Top10['Latitude'], toronto_Top10['Longitude'], toronto_Top10['Neighborhood'], toronto_Top10['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters